In [1]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
import matplotlib.pyplot as plt

ADJ_CLOSE ='Adj Close'

In [2]:
df = pd.read_csv("../data/GOOG.csv")
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-08-19,49.813290,51.835709,47.800831,49.982655,49.982655,44871361
1,2004-08-20,50.316402,54.336334,50.062355,53.952770,53.952770,22942874
2,2004-08-23,55.168217,56.528118,54.321388,54.495735,54.495735,18342897
3,2004-08-24,55.412300,55.591629,51.591621,52.239197,52.239197,15319808
4,2004-08-25,52.284027,53.798351,51.746044,52.802086,52.802086,9232276


수정종가 계산
https://todo-wr-sth.tistory.com/7

In [3]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,4192.000000,4192.000000,4192.000000,4192.000000,4192.000000,4.192000e+03
mean,574.870801,580.540338,569.251095,575.054032,575.054032,6.683645e+06
std,445.672349,450.558610,441.633641,446.319070,446.319070,7.890360e+06
min,49.409801,50.680038,47.800831,49.818268,49.818268,7.922000e+03
25%,242.131180,245.041523,239.814865,242.139904,242.139904,1.660161e+06
50%,366.140121,368.830033,361.004364,365.452683,365.452683,3.995866e+06
75%,819.967499,823.000000,815.617493,819.745011,819.745011,8.512940e+06
max,2277.959961,2289.040039,2257.679932,2285.879883,2285.879883,8.254163e+07


# 볼린저 밴드(Bollinger band)
볼린저 밴드는 현재 주가가 상대적으로 높은지 낮은지를 판단할 때 사용하는 보조지표다.
3개의 선으로 구성되는데, 중심선인 __이동 평균선, 표준 편차의 상단선, 하단선__ 으로 구성된다.
볼린저 밴드를 사용하는 이유는 이동 평균선으로 추세를 관찰하고 상단선과 하단선으로 밴드내에서 현재 가격의 상승과 하락폭을 정량적으로 계산할 수 있기 때문이다.
* 상단 밴드 = 중간 밴드 + 2x20일 이동 표준 편차
* 중간 밴드 = 20일 이동 평균선
* 하단 밴드 = 중간 밴드 - 2x20일 이동 표준 편차

In [4]:
price_df = df.loc[:, ['Date', 'Adj Close']].copy()
price_df.head()

,Date,Adj Close
0,2004-08-19,49.982655
1,2004-08-20,53.952770
2,2004-08-23,54.495735
3,2004-08-24,52.239197
4,2004-08-25,52.802086


In [5]:
price_df.set_index(['Date'], inplace=True)
price_df.head()

,Adj Close
Date,
2004-08-19,49.982655
2004-08-20,53.952770
2004-08-23,54.495735
2004-08-24,52.239197
2004-08-25,52.802086


In [6]:
#이평선 만들기
price_df['center']= price_df['Adj Close'].rolling(window=20).mean()
price_df.iloc[18:25]

,Adj Close,center
Date,,
2004-09-15,55.790882,NaN
2004-09-16,56.772205,52.443678
2004-09-17,58.525631,52.870827
2004-09-20,59.457142,53.146045
2004-09-21,58.699978,53.356258
2004-09-22,58.968971,53.692746
2004-09-23,60.184414,54.061863


In [7]:
#상단선
price_df['ub']=price_df['center'] + 2* price_df['Adj Close'].rolling(window=20).std()
#하단선
price_df['lb']=price_df['center'] - 2* price_df['Adj Close'].rolling(window=20).std()
price_df.iloc[18:25]

,Adj Close,center,ub,lb
Date,,,,
2004-09-15,55.790882,NaN,NaN,NaN
2004-09-16,56.772205,52.443678,56.663844,48.223512
2004-09-17,58.525631,52.870827,57.724062,48.017592
2004-09-20,59.457142,53.146045,58.813589,47.478502
2004-09-21,58.699978,53.356258,59.524354,47.188161
2004-09-22,58.968971,53.692746,60.321328,47.064164
2004-09-23,60.184414,54.061863,61.277773,46.845952


In [8]:
n = 20
sigma = 2
def bollinger_band(price_df, n, sigma):
    bb = price_df.copy()
    bb['center'] = price_df['Adj Close'].rolling(n).mean()
    bb['ub']= bb['center'] + sigma* price_df['Adj Close'].rolling(n).std()
    bb['lb']= bb['center'] - sigma* price_df['Adj Close'].rolling(n).std()
    return bb

bollinger = bollinger_band(price_df, n, sigma)
bollinger[18:25]

,Adj Close,center,ub,lb
Date,,,,
2004-09-15,55.790882,NaN,NaN,NaN
2004-09-16,56.772205,52.443678,56.663844,48.223512
2004-09-17,58.525631,52.870827,57.724062,48.017592
2004-09-20,59.457142,53.146045,58.813589,47.478502
2004-09-21,58.699978,53.356258,59.524354,47.188161
2004-09-22,58.968971,53.692746,60.321328,47.064164
2004-09-23,60.184414,54.061863,61.277773,46.845952


In [9]:
base_date = '2004-01-02'
sample = bollinger.loc[base_date:]
sample.head()

,Adj Close,center,ub,lb
Date,,,,
2004-08-19,49.982655,NaN,NaN,NaN
2004-08-20,53.952770,NaN,NaN,NaN
2004-08-23,54.495735,NaN,NaN,NaN
2004-08-24,52.239197,NaN,NaN,NaN
2004-08-25,52.802086,NaN,NaN,NaN


평균 회귀 전략에서 진입/청산 신호가 발생할 때 우리가 취하는 행동을 기록할 데이터프레임을 만들어본다.(trading book)

In [10]:
book = sample[[ADJ_CLOSE]].copy()
book['trade'] = ''
book.head()

,Adj Close,trade
Date,,
2004-08-19,49.982655,
2004-08-20,53.952770,
2004-08-23,54.495735,
2004-08-24,52.239197,
2004-08-25,52.802086,


In [11]:
def create_trade_book(sample):
    book = sample[[ADJ_CLOSE]].copy()
    book['trade'] = ''
    return (book)

평균 회귀의 기본적인 가정은 현재 주가가 일정기간 평균가격보다 낮으면 매수(미래에 주가가 상승할 것이므로)
반대로 현재 주가가 평균가격보다 높다면 앞으로 주가가 하락할 것으로 예상해 매도한다.
볼린저 밴드 하한선을 돌파할 경우을 과매도 구간이라고 간주하여 매수하고,
상한선을 돌파했을 때는 과매수 구간이라고 간주하여 매도한다.

In [12]:
def tradings(sample, book):
    for i in sample.index:
        if sample.loc[i, ADJ_CLOSE] > sample.loc[i, 'ub']: #상단 밴드 이탈 시 동작 안함
            book.loc[i, 'trade'] = ''
        elif sample.loc[i, 'lb']>sample.loc[i, ADJ_CLOSE]: #하단 밴드 이탈 시 매수
            if book.shift(1).loc[i, 'trade'] == 'buy': #이미 매수상태면
                book.loc[i, 'trade'] = 'buy'
            else:
                book.loc[i, 'trade'] = 'buy'
        elif sample.loc[i, 'ub'] >= sample.loc[i, ADJ_CLOSE] \
        and sample.loc[i, ADJ_CLOSE]>= sample.loc[i, 'lb']: #볼린저 밴드 안에 있을 경우
            if book.shift(1).loc[i, 'trade'] == 'buy':
                book.loc[i, 'trade'] = 'buy'
            else:
                book.loc[i, 'trade'] = ''
    return (book)

In [13]:
book = tradings(sample, book)
book.tail(30)

,Adj Close,trade
Date,,
2021-03-03,2026.709961,
2021-03-04,2049.090088,
2021-03-05,2108.540039,
2021-03-08,2024.170044,
2021-03-09,2052.699951,
2021-03-10,2055.030029,
2021-03-11,2114.770020,
2021-03-12,2061.919922,
2021-03-15,2066.489990,


# 전략 수익률


In [17]:
def returns(book):
    rtn = 1.0
    book['return'] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index:
        #'long 진입(매수)'
        if book.loc[i, 'trade']=='buy' and book.shift(1).loc[i, 'trade'] == '':
            buy = book.loc[i, ADJ_CLOSE]
            print('진입일: {}, long 진입 가격: {}'.format(i, buy))
        #'long 청산(매도)'
        elif book.loc[i, 'trade']==''and book.shift(1).loc[i, 'trade'] == 'buy':
            sell = book.loc[i, ADJ_CLOSE]
            rtn = (sell-buy)/buy+1 #손익 계산
            book.loc[i, 'return'] = rtn
            print('청산일: ', i, 'long 진입 가격: ', buy, ' | long 청산가격: ', sell, '|return:', round(rtn, 4))
        if book.loc[i, 'trade'] == '': #제로 포지션
            buy = 0.0
            sell = 0.0
    acc_rtn = 1.0
    for i in book.index:
        rtn = book.loc[i, 'return']
        acc_rtn = acc_rtn * rtn # 누적 수익율 계산
        book.loc[i, 'acc return']=acc_rtn
    print('Accunulated return: ', acc_rtn)
    return (round(acc_rtn, 4))

In [18]:
print(returns(book))

진입일: 2005-01-24, long 진입 가격: 90.022575
청산일:  2005-02-03 long 진입 가격:  90.022575  | long 청산가격:  105.036301 |return: 1.1668
진입일: 2005-08-22, long 진입 가격: 136.493393
청산일:  2005-09-07 long 진입 가격:  136.493393  | long 청산가격:  146.884445 |return: 1.0761
진입일: 2005-10-13, long 진입 가격: 148.164642
청산일:  2005-10-21 long 진입 가격:  148.164642  | long 청산가격:  169.315369 |return: 1.1428
진입일: 2006-01-20, long 진입 가격: 198.984161
청산일:  2006-03-29 long 진입 가격:  198.984161  | long 청산가격:  196.752518 |return: 0.9888
진입일: 2006-07-20, long 진입 가격: 192.837204
청산일:  2006-09-12 long 진입 가격:  192.837204  | long 청산가격:  195.218277 |return: 1.0123
진입일: 2006-12-18, long 진입 가격: 230.535889
청산일:  2007-01-11 long 진입 가격:  230.535889  | long 청산가격:  248.926956 |return: 1.0798
진입일: 2007-02-05, long 진입 가격: 232.707748
청산일:  2007-04-03 long 진입 가격:  232.707748  | long 청산가격:  235.417603 |return: 1.0116
진입일: 2007-07-23, long 진입 가격: 255.29808
청산일:  2007-09-18 long 진입 가격:  255.29808  | long 청산가격:  266.63559 |return: 1.0444
진입일: 2008-01-04, long